In [2]:
import os
import torch
import wget
import wandb
import tarfile
import itertools
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from abc import ABC, ABCMeta
from retrying import retry

from ogb.utils.url import decide_download, download_url, extract_zip
import ogb.utils.mol as mol
import torch_geometric as tg

In [23]:
class HeteroQM9(tg.data.InMemoryDataset, ABC):
    def __init__(self, root):
        self.url = 'https://figshare.com/ndownloader/files/3195389'
        super().__init__(root)

        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return 'dsgdb9nsd.xyz.tar.bz2'

    @property
    def processed_file_names(self):
        return 'heterogenous_qm9.pt'
    
    def download(self):
        if decide_download(self.url):
            wget.download(self.url, self.raw_paths[0])
        else:
            print('Stop download.')
            exit(-1)

    @staticmethod
    def mol2pyg(molecule):
        graph = mol.smiles2graph(molecule)
        data = tg.data.Data()
        data.__num_nodes__ = int(graph['num_nodes'])
        data.edge_index = torch.from_numpy(graph['edge_index']).to(torch.int64)
        data.edge_attr = torch.from_numpy(graph['edge_feat']).float()
        data.x = torch.from_numpy(graph['node_feat']).float()
        return data
    
    def process(self):
        smiles_prop_dict = {}
        tar = tarfile.open(self.raw_paths[0], "r:bz2")
        for tarinfo in tar:
            if tarinfo.isreg():
                f = tar.extractfile(tarinfo)
                lines = f.read().decode().split('\n')
                print(lines)
                print(lines[1].split('\t')[4:-1])
                print(lines[-3].split('\t')[1])
                print(lines[-4].split('\t'))
                print(lines[-5].split('\t'))
                
                
                
                # exit()
                print(f.read().decode().split('\n')[-3:-4])
                exit()
                smiles = f.read().decode().split('\n')[-3:-4].split('\t')[1]
                y = f.read().decode().split('\n')[-4].split('\t')
                # print(smiles, y)
                # smiles_prop_dict[smiles] = y
                # print(smiles_prop_dict[smiles])
            else:
                continue
        tar.close()
        
        # print(smiles_prop_dict)
        exit()
        

if __name__ == '__main__':
    HeteroQM9(root=os.path.join('GraphCoAttention', 'data'))
        

['5', 'gdb 1\t157.7118\t157.70997\t157.70699\t0.\t13.21\t-0.3877\t0.1171\t0.5048\t35.3641\t0.044749\t-40.47893\t-40.476062\t-40.475117\t-40.498597\t6.469\t', 'C\t-0.0126981359\t 1.0858041578\t 0.0080009958\t-0.535689', 'H\t 0.002150416\t-0.0060313176\t 0.0019761204\t 0.133921', 'H\t 1.0117308433\t 1.4637511618\t 0.0002765748\t 0.133922', 'H\t-0.540815069\t 1.4475266138\t-0.8766437152\t 0.133923', 'H\t-0.5238136345\t 1.4379326443\t 0.9063972942\t 0.133923', '1341.307\t1341.3284\t1341.365\t1562.6731\t1562.7453\t3038.3205\t3151.6034\t3151.6788\t3151.7078', 'C\tC\t', 'InChI=1S/CH4/h1H4\tInChI=1S/CH4/h1H4', '']
['0.', '13.21', '-0.3877', '0.1171', '0.5048', '35.3641', '0.044749', '-40.47893', '-40.476062', '-40.475117', '-40.498597', '6.469']
C
['1341.307', '1341.3284', '1341.365', '1562.6731', '1562.7453', '3038.3205', '3151.6034', '3151.6788', '3151.7078']
['H', '-0.5238136345', ' 1.4379326443', ' 0.9063972942', ' 0.133923']
[]


Processing...


IndexError: list index out of range